In [ ]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (8, 6)

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

<h3>1. Считываем данные по индексу RTSI за годы 2010-2016</h3>

In [ ]:
df = pd.read_csv('data/RTSI_2010-2016.csv')

In [ ]:
df.describe()

<h3>2. Избавляемся от ненужных фич, добавляем пропущенные выходные дни </h3>

In [ ]:
df.drop(['<TIME>', '<HIGH>', '<LOW>', '<VOL>'], axis=1, inplace=True)

In [ ]:
# 0 - цена не изменилась
# 1 - цена выросла
# 2 - цена упала
df['diff'] = (df['<CLOSE>'] - df['<OPEN>']).astype(int)
df['result'] = df['diff'].apply(lambda d: 1 if d > 0 else 2 if d < 0 else 0)

df.drop(['<OPEN>', '<CLOSE>', 'diff'], axis=1, inplace=True)

In [ ]:
df.set_index('<DATE>', inplace=True)

In [ ]:
idx = pd.date_range('01/01/10', '12/31/2016')
df.index = pd.DatetimeIndex(df.index)
df = df.reindex(idx, fill_value=0)

In [ ]:
df = df.reset_index()
df = df.rename(columns = {'index': '<DATE>'})

In [ ]:
df.head()

<h3>3. Форматируем данные</h3>

In [ ]:
df['weekday'] = df['<DATE>'].apply(lambda d: d.weekday())
df['month'] = df['<DATE>'].apply(lambda d: d.month)

df.drop('<DATE>', axis=1, inplace=True)

<h3>4. Смотрим на пару табличек и графиков, построенных по нашим данным</h3>

In [ ]:
df['result'].value_counts()

In [ ]:
df.groupby('result')['weekday', 'month'].describe(percentiles=[])

In [ ]:
pd.crosstab(df['result'], df['weekday'], margins=True)

In [ ]:
sns.countplot(x='weekday', hue='result', data=df);

In [ ]:
df['result'].value_counts().plot(kind='bar', label='result')

In [ ]:
X = df.drop('result', axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
%%time
tsne = TSNE(random_state=17)
tsne_representation = tsne.fit_transform(X_scaled)
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1], 
            c=df['result'].map({0: 'blue', 1: 'orange', 2: 'red'}));

<h3>5. Обучаем две модели, используя алгоритмы kNN и деревья решений</h3>
<h4>Как работает дерево решений?</h4>
<p>Классическое дерево решений как алгоритм машинного обучения объединяет логические правила вида "Значение признака  $a$  меньше  $x$ => одно из искомых значений (0,1,2)". Огромное преимущество деревьев решений в том, что они легко интерпретируемы, понятны человеку. Например, по визуализации нашего дерева видно, что в корне дерева проверяется день недели: если это суббота или воскресенье, то практически наверняка цена не изменится и модель прескажет 0</p>
<h4>Как работает kNN?</h4>
<p>Метод ближайших соседей (k Nearest Neighbors, или kNN) - тоже очень популярный метод классификации. Например, если ты хочешь предсказать результат торгов в субботу, посмотри на другие похожие дни (то есть выходные) и выбери преобладающее среди них значение результата торгов - скорее всего, получишь 0</p>

In [ ]:
y = df['result'].values
df.drop('result', axis=1, inplace=True)

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(df.values, y, test_size=0.3, random_state=17)

tree = DecisionTreeClassifier(max_depth=3, random_state=17)
tree.fit(X_train, y_train)
accuracy_score(y_cv, tree.predict(X_cv))

In [ ]:
tree_params = {'max_depth': range(1,11)}

tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(X_train, y_train)
tree_grid.best_params_

In [ ]:
# draw decision tree
from sklearn.tree import export_graphviz

export_graphviz(tree, feature_names=df.columns.values, 
                out_file='tree.dot', filled=True)
!dot -Tpng tree.dot -o tree.png
!rm tree.dot

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
accuracy_score(y_cv, knn.predict(X_cv))

In [ ]:
from sklearn.pipeline import Pipeline

knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])
knn_params = {'knn__n_neighbors': range(1, 10)}

knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)
knn_grid.best_params_

<h4> Средняя доля правильных ответов на тестовой выборке составляет примерно 60%</h4>

<h3>6. Теперь с помощью обученных моделей предскажем поведение индекса RTSI в 2017 году</h3>

In [ ]:
test_df = pd.read_csv('data/RTSI_2017.csv')
test_df.drop(['<TIME>', '<HIGH>', '<LOW>', '<VOL>'], axis=1, inplace=True)

# 0 - цена не изменилась
# 1 - цена выросла
# 2 - цена упала
test_df['diff'] = (test_df['<CLOSE>'] - test_df['<OPEN>']).astype(int)
test_df['result'] = test_df['diff'].apply(lambda d: 1 if d > 0 else 2 if d < 0 else 0)
test_df.drop(['<OPEN>', '<CLOSE>', 'diff'], axis=1, inplace=True)

test_df.set_index('<DATE>', inplace=True)
idx = pd.date_range('01/01/17', '09/26/17')
test_df.index = pd.DatetimeIndex(test_df.index)
test_df = test_df.reindex(idx, fill_value=0)
test_df = test_df.reset_index()
test_df = test_df.rename(columns = {'index': '<DATE>'})

test_df['weekday'] = test_df['<DATE>'].apply(lambda d: d.weekday())
test_df['month'] = test_df['<DATE>'].apply(lambda d: d.month)
test_df.drop('<DATE>', axis=1, inplace=True)

y_test = test_df['result'].values
test_df.drop('result', axis=1, inplace=True)

accuracy_score(y_test, tree.predict(test_df.values))
accuracy_score(y_test, knn.predict(test_df.values))

<h4> Средняя доля правильных ответов на этой выборке также составляет примерно 60%</h4>